# Objects Detection

> - 🤖 See [full list of Machine Learning Experiments](https://github.com/trekhleb/machine-learning-experiments) on GitHub
> - ▶️ Demo: [try this model in action](https://trekhleb.github.io/machine-learning-experiments)

## Importing Dependencies

- [tensorflow](https://www.tensorflow.org/) - for developing and training ML models.
- [matplotlib](https://matplotlib.org/) - for plotting the data.
- [numpy](https://numpy.org/) - for linear algebra operations.

In [95]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import pathlib

print('Tensorflow version:', tf.__version__)
print('Keras version:', tf.keras.__version__)

Tensorflow version: 2.1.0
Keras version: 2.2.4-tf


## Loading the model

To do objects detection we're going to use [ssdlite_mobilenet_v2_coco](http://download.tensorflow.org/models/object_detection/ssdlite_mobilenet_v2_coco_2018_05_09.tar.gz) model from [Tensorflow detection models zoo](https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/detection_model_zoo.md).

In [111]:
# Create cache folder.
!mkdir .tmp

mkdir: .tmp: File exists


In [112]:
def load_model(model_name):
    model_url = 'http://download.tensorflow.org/models/object_detection/' + model_name + '.tar.gz'
    
    model_dir = tf.keras.utils.get_file(
        fname=model_name, 
        origin=model_url,
        untar=True,
        cache_dir=pathlib.Path('.tmp').absolute()
    )
    model = tf.saved_model.load(model_dir + '/saved_model')
    
    return model

In [121]:
model_name = 'ssdlite_mobilenet_v2_coco_2018_05_09'
saved_model = load_model(model_name)

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


In [122]:
# Exploring model signatures.
saved_model.signatures

_SignatureMap({'serving_default': <tensorflow.python.eager.wrap_function.WrappedFunction object at 0x16f1f9e50>})

In [124]:
# Loading default model signature.
model = saved_model.signatures['serving_default']

In [23]:
# Load the TensorBoard notebook extension.
# %reload_ext tensorboard
%load_ext tensorboard

# Front-End

On the FE we're using `@tensorflow-models/coco-ssd`

- https://www.npmjs.com/package/@tensorflow-models/coco-ssd
- https://github.com/tensorflow/tfjs-models#readme
- https://github.com/tensorflow/tfjs-models/tree/master/coco-ssd

Model: `COCO-SSD`

Base model: `lite_mobilenet_v2`

## Converting the model to Web

```
tensorflowjs_converter --input_format=tf_frozen_model \
                       --output_format=tfjs_graph_model \
                       --output_node_names='Postprocessor/ExpandDims_1,Postprocessor/Slice' \
                       ./frozen_inference_graph.pb \
                       ./web_model
```


# COCO Trained Models

https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/detection_model_zoo.md#coco-trained-models

**ssdlite_mobilenet_v2_coco**

http://download.tensorflow.org/models/object_detection/ssdlite_mobilenet_v2_coco_2018_05_09.tar.gz

# Try

In [40]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import pathlib
from PIL import Image

print('Tensorflow version:', tf.__version__)
print('Keras version:', tf.keras.__version__)

Tensorflow version: 2.1.0
Keras version: 2.2.4-tf


In [9]:
model_path = '.tmp/ssdlite_mobilenet_v2_coco_2018_05_09/saved_model'

In [10]:
model = tf.saved_model.load(model_path)

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


In [11]:
model

In [12]:
model.signatures

_SignatureMap({'serving_default': <tensorflow.python.eager.wrap_function.WrappedFunction object at 0x143eb4d90>})

In [13]:
model_signature = model.signatures['serving_default']

In [20]:
# This line is slow and heavy but contains a lot of info about the model.
# model_signature.function_def

In [21]:
model_signature.output_shapes

{'detection_scores': TensorShape([None, 100]),
 'detection_classes': TensorShape([None, 100]),
 'num_detections': TensorShape([None]),
 'detection_boxes': TensorShape([None, 100, 4])}

In [35]:
PATH_TO_TEST_IMAGES_DIR = pathlib.Path('./images')
TEST_IMAGE_PATHS = sorted(list(PATH_TO_TEST_IMAGES_DIR.glob("*.jpg")))
TEST_IMAGE_PATHS

[PosixPath('images/01.jpg'),
 PosixPath('images/02.jpg'),
 PosixPath('images/03.jpg'),
 PosixPath('images/04.jpg'),
 PosixPath('images/05.jpg'),
 PosixPath('images/06.jpg'),
 PosixPath('images/07.jpg')]

In [43]:
image_np = np.array(Image.open(TEST_IMAGE_PATHS[0]))
image_np

array([[[244, 241, 236],
        [242, 242, 234],
        [248, 250, 239],
        ...,
        [ 27,  27,  27],
        [ 25,  25,  27],
        [ 29,  28,  33]],

       [[255, 252, 249],
        [247, 246, 241],
        [243, 244, 236],
        ...,
        [ 21,  21,  21],
        [ 21,  21,  23],
        [ 23,  22,  27]],

       [[255, 253, 252],
        [238, 237, 233],
        [219, 220, 214],
        ...,
        [ 25,  25,  25],
        [ 24,  24,  26],
        [ 21,  21,  23]],

       ...,

       [[179, 176, 169],
        [177, 174, 167],
        [174, 171, 164],
        ...,
        [113, 110, 103],
        [113, 110, 103],
        [113, 110, 103]],

       [[183, 180, 173],
        [181, 178, 171],
        [178, 175, 168],
        ...,
        [107, 104,  97],
        [106, 103,  96],
        [107, 104,  97]],

       [[184, 181, 174],
        [182, 179, 172],
        [180, 177, 170],
        ...,
        [105, 102,  95],
        [104, 101,  94],
        [104, 101,  94]]

In [51]:
input_tensor = tf.convert_to_tensor(image_np)
input_tensor

<tf.Tensor: shape=(768, 1024, 3), dtype=uint8, numpy=
array([[[244, 241, 236],
        [242, 242, 234],
        [248, 250, 239],
        ...,
        [ 27,  27,  27],
        [ 25,  25,  27],
        [ 29,  28,  33]],

       [[255, 252, 249],
        [247, 246, 241],
        [243, 244, 236],
        ...,
        [ 21,  21,  21],
        [ 21,  21,  23],
        [ 23,  22,  27]],

       [[255, 253, 252],
        [238, 237, 233],
        [219, 220, 214],
        ...,
        [ 25,  25,  25],
        [ 24,  24,  26],
        [ 21,  21,  23]],

       ...,

       [[179, 176, 169],
        [177, 174, 167],
        [174, 171, 164],
        ...,
        [113, 110, 103],
        [113, 110, 103],
        [113, 110, 103]],

       [[183, 180, 173],
        [181, 178, 171],
        [178, 175, 168],
        ...,
        [107, 104,  97],
        [106, 103,  96],
        [107, 104,  97]],

       [[184, 181, 174],
        [182, 179, 172],
        [180, 177, 170],
        ...,
        [105, 102,  

In [52]:
input_tensor = input_tensor[tf.newaxis, ...]
input_tensor

<tf.Tensor: shape=(1, 768, 1024, 3), dtype=uint8, numpy=
array([[[[244, 241, 236],
         [242, 242, 234],
         [248, 250, 239],
         ...,
         [ 27,  27,  27],
         [ 25,  25,  27],
         [ 29,  28,  33]],

        [[255, 252, 249],
         [247, 246, 241],
         [243, 244, 236],
         ...,
         [ 21,  21,  21],
         [ 21,  21,  23],
         [ 23,  22,  27]],

        [[255, 253, 252],
         [238, 237, 233],
         [219, 220, 214],
         ...,
         [ 25,  25,  25],
         [ 24,  24,  26],
         [ 21,  21,  23]],

        ...,

        [[179, 176, 169],
         [177, 174, 167],
         [174, 171, 164],
         ...,
         [113, 110, 103],
         [113, 110, 103],
         [113, 110, 103]],

        [[183, 180, 173],
         [181, 178, 171],
         [178, 175, 168],
         ...,
         [107, 104,  97],
         [106, 103,  96],
         [107, 104,  97]],

        [[184, 181, 174],
         [182, 179, 172],
         [180, 17

In [62]:
output_dict = model_signature(input_tensor)
output_dict

{'detection_scores': <tf.Tensor: shape=(1, 100), dtype=float32, numpy=
 array([[0.9872208 , 0.97772163, 0.97706723, 0.33399296, 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        ,

In [66]:
# num_detections = int(output_dict.pop('num_detections'))
num_detections = int(output_dict['num_detections'])
num_detections

4

In [77]:
# output_dict_np = {key:value[0, :num_detections].numpy() for key, value in output_dict.items()}
output_dict_np = {key:value[0, :num_detections].numpy() for key, value in output_dict.items() if key != 'num_detections'}
output_dict_np['num_detections'] = num_detections
output_dict_np['detection_classes'] = output_dict_np['detection_classes'].astype(np.int64)
output_dict_np

{'detection_scores': array([0.9872208 , 0.97772163, 0.97706723, 0.33399296], dtype=float32),
 'detection_classes': array([3, 1, 2, 3]),
 'detection_boxes': array([[0.31191874, 0.14469081, 0.5815682 , 0.5780536 ],
        [0.16846958, 0.47724158, 0.87531245, 0.75766236],
        [0.48911124, 0.37843052, 0.9204666 , 0.90651405],
        [0.30284303, 0.        , 0.53111345, 0.10418391]], dtype=float32),
 'num_detections': 4}

**Labels**

https://github.com/tensorflow/models/blob/master/research/object_detection/data/mscoco_label_map.pbtxt

In [78]:
PATH_TO_LABELS = '.tmp/mscoco_label_map.pbtxt'

In [79]:
with tf.gfile.GFile(PATH_TO_LABELS, 'r') as fid:
    label_map_string = fid.read()
    print(label_map_string)

AttributeError: module 'tensorflow' has no attribute 'gfile'